In [2]:
import lutorpy as lua
import numpy as np
import pandas as pd
from PIL import Image

require("torch")
require('nn')
require('cunn')
# lua.LuaRuntime(zero_based_index=False)

True

In [3]:
def ReshapeImage(test):
    im = Image.fromarray(test)
    im = im.convert('RGB')
    im_c = im.resize((24,32), 1)
    return np.array(im_c).transpose(2,0,1)

# def predict(model, array):
#     image = np.array([i.astype(np.float64) for i in array])
#     tensor_image = torch.fromNumpyArray(image)
#     yhat = model._forward(tensor_image)
#     return yhat.asNumpyArray()

def normalize(ii, norm):
    _temp = np.zeros(ii.shape)
    for n in norm.keys():
        _temp[n] = (ii[n] - norm[n]['mean']) / norm[n]['std']
    return _temp

In [4]:
# -- load a model
model = torch.load('/home/rnam/Documents/ped/notebook/test_model.t7')
model._get(20)._add(nn.LogSoftMax())._add(nn.Exp())
# model._add(nn.LogSoftMax())
# model._add(nn.Exp())

# -- load the mean/std
# -- recenter
params = torch.load('/home/rnam/Documents/ped/run/output/20160710_full_aug_balanced_m7.net')
norm = {i:{'mean':0.0,'std':0.0} for i in range(3)}
for j in range(3):
    norm[j]['mean'] = params['mean'][j]
    norm[j]['std'] = params['std'][j]
    
# -- import the data from pickle and transform data
df = pd.read_pickle('/home/rnam/Documents/ped/data/20160708_snapshot/patch2.pkl')
norm_images = np.array([normalize(ReshapeImage(i),norm) for i in df.patch.values])

In [5]:
# df2 = df[:1000]
# im2 = norm_images[:1000]

In [18]:
# -- test the speed
def p2():
    interval = 3
    n = 1000
    scores = np.zeros(n)
    for i in np.arange(0,n,interval):
        prange = np.arange(i, np.minimum(i+interval,n))
        y = predict(model, norm_images[prange])
        for j, v in enumerate(prange):
            scores[v] = np.argmax(y[j])
    return scores

In [20]:
%timeit p2()

1 loop, best of 3: 1min 6s per loop


In [ ]:
scores